In [ ]:
import requests
import os
import json

VOYAGE_API_KEY = os.getenv('VOYAGE_API_KEY', 'your_api_key_here')

url = 'https://api.voyageai.com/v1/embeddings'
headers = {
    'Authorization': f'Bearer {VOYAGE_API_KEY}',
    'Content-Type': 'application/json'
}

In [ ]:
import pickle
with open('./data/comments.pkl', 'rb') as handle:
    comments = (list(set(pickle.load(handle))))

In [ ]:
from tqdm import tqdm
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

comment_embedding_map = {}
for chunk in tqdm(chunker(comments,128)):
    data = {
        "input": chunk,
        "model": "voyage-large-2-instruct"
    }
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        results = response.json()

        for i, text in enumerate(chunk):
            comment_embedding_map[text] = results['data'][i]['embedding']
    except Exception as e:
        print(e)
        for i, text in enumerate(chunk):
            comment_embedding_map[text] = None

In [ ]:
with open('./data/comments_embeddings_voyage_ai.pkl', 'wb') as handle:
    pickle.dump(comment_embedding_map, handle, protocol=pickle.HIGHEST_PROTOCOL)